In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Reshape, Attention, TimeDistributed, Embedding, LSTM, Bidirectional, Dropout, Dense, GRU, concatenate, Conv1D, Conv2D, Flatten,MaxPooling1D, MaxPooling2D, LocallyConnected1D, Activation, GaussianNoise, BatchNormalization, RepeatVector
from tensorflow.keras.models import Model, Sequential
import nltk
from nltk.corpus import stopwords
from AttentionWeightedAverage import AttentionWeightedAverage
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

- NLP on Title + Description to Tags, in order to build a tag recommendation algorithm
- NLP on Tags + Description + Category to Title, in order to provide title recommendation algorithm. These videos are popular videos, meaning they have appealing titles. Let's build a system, that judging by content, can provide the most appropriate title.

In [2]:
df = pd.read_csv("./data/USvideos.csv")
print('Number of youtube videos in dataframe: ', len(df.index))
df.head()

# count category occurences
occurences = df['category_id'].value_counts()
# category names
map_categories = {}
with open('data/US_category_id.json') as json_file:
    data_category = json.load(json_file)
    for category in data_category['items']:
        int_id = int(category['id'])
        occ = 0
        if int_id in occurences:
            occ = occurences[int_id]
        map_categories[int_id] = [category['snippet']['title'],occ]
# create new dataframe with categories and appearances
df_categories = pd.DataFrame.from_dict(map_categories, orient='index', columns=['category_name', 'count'])
# add category name on the main dataframe
for i in map_categories.keys():
    df.loc[df['category_id'] == i,'category_name'] = map_categories[i][0]

Number of youtube videos in dataframe:  40949


### Tags, Description and Category -> Auto generate Title
We will see a word level seq2seq model

In [3]:
def discretize(tags):
    tags = tags.replace('"','')
    discrete_tags = tags.split('|')
    return discrete_tags

In [4]:
df_text = df[['title', 'description', 'tags', 'category_name']]
df_text = df_text.dropna(axis=0, how='any')

titles = df_text['title'].tolist()
descriptions = df_text['description'].tolist()
tags = [discretize(row['tags']) for index, row in df_text.iterrows()]
categories = df_text['category_name'].tolist()

- Titles need to be tokenized, lowercased, categorical.
- Descriptions need to be tokenized, lowercased, categorical.
- Tags need to be lowercased, categorical.

In [5]:
stops = set(stopwords.words("english"))

In [6]:
clean_desc = []
clean_title = []
for desc in descriptions:
    desc = desc.replace('\\n', ' ')
    desc = desc.lower()
    desc = desc.replace('www.facebook.com', 'FacebookLink')
    desc = desc.replace('http.facebook.com', 'FacebookLink')
    desc = desc.replace('www.twitter.com', 'TwitterLink')
    desc = desc.replace('www.amazon.com', 'AmazonLink')
    desc = desc.replace('http.amzn.com', 'AmazonLink')
    desc = desc.replace('www.instagram.com', 'InstagramLink')
    desc = desc.replace('www.snapchat.com', 'SnapchatLink')
    desc = desc.replace('https://bit.ly', 'PortalLink')
    temp = text_to_word_sequence(desc, filters='!"#%&()*-+,.\\/:;<=>?@[\\]^_`{|}~\t\n♡▶➜', lower=True, split=' ')
    for w in temp:
        if w in stops:
            temp.remove(w)
    clean_desc.append(temp)
    
for title in titles:
    title = title.replace('\\n', ' ')
    title = title.replace('\xa0', ' ')
    seq = text_to_word_sequence(title, filters='"#%&()*-+,.\\/:;<=>?@[\\]^_`{|}~\t\n♡▶➜', lower=True, split=' ')
    clean_title.append(['START_ '] + seq + [' _END'])

Training an unsupervised method for word2vec with gensim, in order to provide me with very good embeddings

In [7]:
from gensim.models import Word2Vec
input_features = 100

gensim_model = Word2Vec(size=input_features,
                       window=5,
                       min_count=1,
                       sg=1,
                       hs=0,
                       negative=10,
                       workers=4)
gensim_model.build_vocab(clean_desc+clean_title+tags)
gensim_model.train(clean_desc+clean_title+tags, total_examples=len(clean_desc+clean_title+tags), epochs=10)

(57577047, 64014370)

Compute the vocabulary for the words that appear in the titles

In [8]:
all_title_words=set()
for title in clean_title:
    for word in title:
        if word not in all_title_words:
            all_title_words.add(word)

The encoder and decoder tokens defer, the encoder takes words from the description, while the decoder produces words from the titles

In [9]:
num_encoder_tokens = len(gensim_model.wv.vocab)

target_words = sorted(list(all_title_words))
num_decoder_tokens = len(all_title_words)

Creating my new sequences with vector representations of words

In [10]:
X_train_description = []
X_train_category = []
X_train_tags = []

lb_cat = LabelBinarizer()
lb_cat.fit(list(set(categories)))

maxLenDesc = 0
for desc in clean_desc:
    X_train_description.append(np.asarray([gensim_model.wv[word] for word in desc]))
    if len(desc) > maxLenDesc:
        maxLenDesc = len(desc)
        
maxLenTags = 0
for taglist in tags:
    X_train_tags.append(np.asarray([gensim_model.wv[tag] for tag in taglist]))
    if len(taglist) > maxLenTags:
        maxLenTags = len(taglist)

for cat in categories:
    X_train_category.append(lb_cat.transform([cat]))

maxLenTitle = 0
for title in clean_title:
    if len(title) > maxLenTitle:
        maxLenTitle = len(title)

Create the decoder inputs and outputs. Decoder target data will be ahead by one timestep, and will not include the start character.

In [11]:
X_train_title = np.zeros(
    (len(clean_title), maxLenTitle, input_features),
    dtype='float32')

Y_train_title = np.zeros(
    (len(clean_title), maxLenTitle, num_decoder_tokens),
    dtype='float32')

# generate data
for i, target_text in enumerate(clean_title):
    for t, word in enumerate(target_text):
        # decoder input data, will have the embeddings of the words
        X_train_title[i, t] = gensim_model.wv[word]
        if t > 0:
            # decoder target data, will be ahead by one timestep, and will contain
            # one hot representations of what word is selected
            Y_train_title[i, t - 1, target_words.index(word)] = 1.

Pad Sequences

In [12]:
# padding
X_train_description = pad_sequences(X_train_description, maxlen=maxLenDesc, padding='post')
X_train_tags = pad_sequences(X_train_tags, maxlen=maxLenTags, padding='post')

# convert to numpy arrays
X_train_description = np.stack(X_train_description, axis=0)
X_train_category = np.stack(X_train_category, axis=0)
X_train_tags = np.stack(X_train_tags, axis=0)

Train - Test Split
* Encoder inputs
* Decoder inputs + outputs

In [13]:
# encoder + decoder inputs
X_train_description, X_test_description, X_train_title, X_test_title = train_test_split(X_train_description, X_train_title, test_size=0.05, shuffle=False)

In [26]:
num_titles = len(clean_title)
test_size = X_test_description.shape[0]
# decoder outputs
Y_train_title, Y_test_title = (Y_train_title[:num_titles-test_size], Y_train_title[num_titles-test_size:num_titles])

### Sequence to sequence model

In [15]:
def Seq2SeqModel(vocab_size,input_max_seq,output_target_seq,input_features):
    #encoder
    encoder_input = Input(shape=(input_max_seq,input_features), name='main_input')
    encoder_lstm = LSTM(100, return_state=True, name='encoder_lstm')
    encoder_outputs, encoder_state_h, encoder_state_c = encoder_lstm(encoder_input)
    encoder_states = [encoder_state_h, encoder_state_c]
    
    #decoder
    decoder_input = Input(shape=(None,input_features))
    decoder_lstm = LSTM(100, return_sequences=True, return_state=True, name='decoder_lstm')
    decoder_outputs, decoder_state_h, decoder_state_c = decoder_lstm(decoder_input, initial_state=encoder_states)
    
    decoder_dense = Dense(vocab_size, name='decoder_dense', activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    
    model = Model(inputs=[encoder_input, decoder_input], outputs=decoder_outputs)
    # compile model
    model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=4e-3), metrics=['accuracy'])
    # encoder model
    encoder_model = Model(encoder_input, encoder_states)

    decoder_state_inputs = [Input(shape=(100,)), Input(shape=(100,))]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_input, initial_state=decoder_state_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_input] + decoder_state_inputs, [decoder_outputs] + decoder_states)
    
    return model,encoder_model,decoder_model

In [18]:
epochs=2
batch_size=128
model,encoder_model,decoder_model = Seq2SeqModel(num_decoder_tokens,maxLenDesc,maxLenTitle,100)
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         [(None, 722, 100)]   0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            [(None, None, 100)]  0                                            
__________________________________________________________________________________________________
encoder_lstm (LSTM)             [(None, 100), (None, 80400       main_input[0][0]                 
__________________________________________________________________________________________________
decoder_lstm (LSTM)             [(None, None, 100),  80400       input_7[0][0]                    
                                                                 encoder_lstm[0][1]         

In [31]:
model.fit([X_train_description, X_train_title], Y_train_title, epochs=epochs, batch_size=batch_size,verbose=1)

Train on 38360 samples
Epoch 1/2
38360/38360 [==============================] - 812s 21ms/sample - loss: 2.8617 - accuracy: 0.6146
Epoch 2/2
38360/38360 [==============================] - 855s 22ms/sample - loss: 2.2052 - accuracy: 0.6242


In [100]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = gensim_model.wv['START_ ']
    target_seq = target_seq.reshape((1,1,target_seq.shape[0]))
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq, states_value[0], states_value[1]])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = target_words[sampled_token_index]
        decoded_sentence += ' ' + sampled_word

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_word == ' _END' or len(decoded_sentence) > maxLenTitle):
            stop_condition = True

        target_seq = np.append(target_seq, gensim_model.wv[sampled_word].reshape((1,1,input_features)), axis=1)
        # Update states
        states_value = [h, c]

    return decoded_sentence

In [106]:
i = 0
for i in list(range(10)):
    title_generated = decode_sequence(X_test_description[i].reshape((1,X_test_description.shape[1],X_test_description.shape[2])))
    print("i: ", i)
    print("Description:\n",descriptions[i])
    print("\nGenerated Title: ", title_generated)
    print("\n\n")
    i+=1

i:  0
Description:
 SHANTELL'S CHANNEL - https://www.youtube.com/shantellmartin\nCANDICE - https://www.lovebilly.com\n\nfilmed this video in 4k on this -- http://amzn.to/2sTDnRZ\nwith this lens -- http://amzn.to/2rUJOmD\nbig drone - http://tinyurl.com/h4ft3oy\nOTHER GEAR ---  http://amzn.to/2o3GLX5\nSony CAMERA http://amzn.to/2nOBmnv\nOLD CAMERA; http://amzn.to/2o2cQBT\nMAIN LENS; http://amzn.to/2od5gBJ\nBIG SONY CAMERA; http://amzn.to/2nrdJRO\nBIG Canon CAMERA; http://tinyurl.com/jn4q4vz\nBENDY TRIPOD THING; http://tinyurl.com/gw3ylz2\nYOU NEED THIS FOR THE BENDY TRIPOD; http://tinyurl.com/j8mzzua\nWIDE LENS; http://tinyurl.com/jkfcm8t\nMORE EXPENSIVE WIDE LENS; http://tinyurl.com/zrdgtou\nSMALL CAMERA; http://tinyurl.com/hrrzhor\nMICROPHONE; http://tinyurl.com/zefm4jy\nOTHER MICROPHONE; http://tinyurl.com/jxgpj86\nOLD DRONE (cheaper but still great);http://tinyurl.com/zcfmnmd\n\nfollow me; on http://instagram.com/caseyneistat\non https://www.facebook.com/cneistat\non https://twitter.

i:  8
Description:
 Inspired by the imagination of P.T. Barnum, The Greatest Showman is an original musical that celebrates the birth of show business & tells of a visionary who rose from nothing to create a spectacle that became a worldwide sensation.\n\nIn Theaters This Christmas\n\nDirected By Michael Gracey\nCast: Hugh Jackman, Michelle Williams, Zac Efron, Zendaya, Rebecca Ferguson\n\nSUBSCRIBE: http://bit.ly/FOXSubscribe\n\n\nVisit the Official Site Here: http://www.foxmovies.com/movies/the-greatest-showman\nLike The Greatest Showman on FACEBOOK: http://fox.co/GreatestShowmanFB\nFollow The Greatest Showman on TWITTER: http://fox.co/GreatestShowmanTW\nFollow The Greatest Showman on INSTAGRAM: http://fox.co/GreatestShowmanIG\n\n#GreatestShowman\n\nAbout 20th Century FOX:\nOfficial YouTube Channel for 20th Century Fox Movies. Home of Avatar, Aliens, X-Men, Die Hard, Deadpool, Ice Age, Alvin and the Chipmunks, Rio, Peanuts, Maze Runner, Planet of the Apes, Wolverine and many more.\n\